In [96]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [97]:
file_path = Path("../Resources/cleaned_data.csv")
df= pd.read_csv(file_path)

In [98]:
df['delivery'] = df['transactions'].str.contains('delivery', case=False, na=False)
df['pickup'] = df['transactions'].str.contains('pickup', case=False, na=False)
df['restaurant_reservation'] = df['transactions'].str.contains('restaurant_reservation', case=False, na=False)

# Fill NaN values with False in the new columns
df['delivery'] = df['delivery'].fillna(False)
df['pickup'] = df['pickup'].fillna(False)
df['restaurant_reservation'] = df['restaurant_reservation'].fillna(False)

In [99]:
def is_expensive(price):
    return price > 2

# Create a new 'success' column based on the 'rating' column
df['pricey'] = df['price'].apply(is_expensive)

df.head()

,id,name,image_url,is_closed,url,review_count,rating,transactions,price,group_city,cuisines,latitude,longitude,state,delivery,pickup,restaurant_reservation,pricey
0,a0IET3_yCFcO36OqGSsisg,Eataly NYC Flatiron,https://s3-media4.fl.yelpcdn.com/bphoto/1UDlnu...,False,https://www.yelp.com/biz/eataly-nyc-flatiron-n...,6101,4.0,"delivery, pickup",2,New York City,Italian,40.742101,-73.989922,NY,True,True,False,False
1,zj8Lq1T8KIC5zwFief15jg,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5028,4.5,"delivery, pickup",1,New York City,Italian,40.723088,-73.994530,NY,True,True,False,False
2,16ZnHpuaaBt92XWeJHCC5A,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4828,4.5,"pickup, delivery",2,New York City,Italian,40.733798,-73.999774,NY,True,True,False,False
3,vyoA8dxwScuMV_AsTcjQcg,L & B Spumoni Gardens,https://s3-media1.fl.yelpcdn.com/bphoto/hN5xKw...,False,https://www.yelp.com/biz/l-and-b-spumoni-garde...,4648,4.0,"delivery, pickup",2,New York City,Italian,40.594715,-73.981316,NY,True,True,False,False
4,22nKUyCIbpnzR6R3_g1ptQ,Carmine's Italian Restaurant - Times Square,https://s3-media1.fl.yelpcdn.com/bphoto/0UszeE...,False,https://www.yelp.com/biz/carmines-italian-rest...,4640,4.0,"delivery, pickup",2,New York City,Italian,40.757498,-73.986653,NY,True,True,False,False


In [100]:
y = df['pricey']
x = df.drop(columns=['price','pricey','image_url', 'url', 'id', 'name']) 


In [101]:
from sklearn import preprocessing
from sklearn import utils

lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

In [102]:
X= x.fillna(0)

In [103]:
X.head(20)

,is_closed,review_count,rating,transactions,group_city,cuisines,latitude,longitude,state,delivery,pickup,restaurant_reservation
0,False,6101,4.0,"delivery, pickup",New York City,Italian,40.742101,-73.989922,NY,True,True,False
1,False,5028,4.5,"delivery, pickup",New York City,Italian,40.723088,-73.994530,NY,True,True,False
2,False,4828,4.5,"pickup, delivery",New York City,Italian,40.733798,-73.999774,NY,True,True,False
3,False,4648,4.0,"delivery, pickup",New York City,Italian,40.594715,-73.981316,NY,True,True,False
4,False,4640,4.0,"delivery, pickup",New York City,Italian,40.757498,-73.986653,NY,True,True,False
5,False,3812,4.5,delivery,New York City,Mexican,40.742255,-74.005958,NY,True,False,False
6,False,3317,4.0,"pickup, delivery",New York City,Mexican,40.722930,-73.994190,NY,True,True,False
7,False,3314,4.0,"delivery, pickup",New York City,Italian,40.704930,-73.933990,NY,True,True,False
8,False,3150,4.5,pickup,New York City,Italian,40.722766,-73.996233,NY,False,True,False
9,False,3138,3.5,"delivery, pickup",New York City,Italian,40.760830,-73.989810,NY,True,True,False


In [104]:
X =pd.get_dummies(X)
X.head()

,is_closed,review_count,rating,latitude,longitude,delivery,pickup,restaurant_reservation,transactions_0,transactions_delivery,...,state_ON,state_OR,state_PA,state_PE,state_SC,state_TN,state_TX,state_VA,state_WA,state_WI
0,False,6101,4.0,40.742101,-73.989922,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,5028,4.5,40.723088,-73.994530,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,4828,4.5,40.733798,-73.999774,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,4648,4.0,40.594715,-73.981316,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,4640,4.0,40.757498,-73.986653,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, random_state=42)

In [106]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

In [107]:
model2 = tree.DecisionTreeRegressor()

In [108]:
model2 = model2.fit(X_train_scaled, y_train)

In [109]:
predictions = model2.predict(X_test_scaled)

In [110]:
from sklearn import metrics
import numpy as np


# Calculate error metrics
mae = metrics.mean_absolute_error(y_test, predictions)
mse = metrics.mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

Mean Absolute Error (MAE): 0.07
Mean Squared Error (MSE): 0.07
Root Mean Squared Error (RMSE): 0.26


In [111]:
model = tree.DecisionTreeClassifier()

In [112]:
model = model.fit(X_train_scaled, y_train)

In [113]:
class_predictions = model.predict(X_test_scaled)

In [114]:
print(classification_report(y_test, class_predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      4579
           1       0.20      0.18      0.19       208

    accuracy                           0.93      4787
   macro avg       0.58      0.57      0.58      4787
weighted avg       0.93      0.93      0.93      4787

